# 2D mesh to validate the advection-diffusion solver

This mesh represents two rectangles adjacent to each other, with the rectangle on the left having a slightly higher height than the rectangle on the right. The left rectangle represents water flowing vertically from top to bottom at a certain velocity **q**. The right rectangle represents metal. The objective is to study the heat transmission through the system when the incoming water temperature is higher than the initial temperature of the system, for different values of **q**. An energy balance is also performed to verify that energy is conserved within the system (no energy losses at the interface between the two materials, notably).

## Define geometry

In [1]:
import gmsh
import numpy as np
from math import cos, sin, pi

# initialize the model
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)

gmsh.model.add("2D_diffusion_advection")

L = 2
h = 2
dh = 0.1

# Define points
geom = gmsh.model.geo
p1 = geom.addPoint(0, 0, 0)
p2 = geom.addPoint(L/2, 0, 0)
p3 = geom.addPoint(L, 0, 0)
p4 = geom.addPoint(L, h, 0)
p5 = geom.addPoint(L/2, h, 0)
p6 = geom.addPoint(L/2, h+dh, 0)
p7 = geom.addPoint(0, h+dh, 0)

# Define lines from points
l1 = geom.addLine(p1, p2)
l2 = geom.addLine(p2, p3)
l3 = geom.addLine(p3, p4)
l4 = geom.addLine(p4, p5)
l5 = geom.addLine(p5, p6)
l6 = geom.addLine(p6, p7)
l7 = geom.addLine(p7, p1)
l8 = geom.addLine(p2, p5)

# Create curve loops
cl_left = geom.addCurveLoop([l1, l8, l5, l6, l7])
cl_right = geom.addCurveLoop([l2, l3, l4, -l8])
    
# Define surfaces
s_left = geom.addPlaneSurface([cl_left])
s_right = geom.addPlaneSurface([cl_right])

geom.synchronize()

## Define surfaces for materials

In [2]:
# Fluid domain
gmsh.model.addPhysicalGroup(2, [s_left], 101)
gmsh.model.setPhysicalName(2, 101, 'fluid')

# Rock domain
gmsh.model.addPhysicalGroup(2, [s_right], 102)
gmsh.model.setPhysicalName(2, 102, 'steel')

## Define curves for boundary conditions and energy calculations

In [3]:
gmsh.model.addPhysicalGroup(1, [l6], 1)
gmsh.model.setPhysicalName(1, 1, 'inlet')

gmsh.model.addPhysicalGroup(1, [l1], 2)
gmsh.model.setPhysicalName(1, 2, 'outlet')

## Generate mesh

In [4]:
# Define the mesh element size
lc = 0.01
gmsh.model.mesh.setSize(gmsh.model.getEntities(), lc)

gmsh.model.mesh.generate(2)

# Save mesh
filename = '2D_mesh.msh'
gmsh.write(filename)

gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 40%] Meshing curve 4 (Line)
Info    : [ 50%] Meshing curve 5 (Line)
Info    : [ 70%] Meshing curve 6 (Line)
Info    : [ 80%] Meshing curve 7 (Line)
Info    : [ 90%] Meshing curve 8 (Line)
Info    : Done meshing 1D (Wall 0.000717411s, CPU 0.001225s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 1.48758s, CPU 1.4743s)
Info    : 48023 nodes 96251 elements
Info    : Writing '2D_mesh.msh'...
Info    : Done writing '2D_mesh.msh'
